## Övning #5
### 25 September 2020

#### Agenda för idag:
- Test 5
- fortsättning av klasser
    - `Laboartion 6` kommer handla om klasser
    - magic methods
    - privata / publika metoder och variabler
- förstå oss på lambda (`kan komma att användas i laboration 6`)
- Felhantering


## Klasser (fortsättning)

In [ ]:
# Klasser exempel #4
import operator # En inbyggd modul som hjälper med att dynamiskt ange aritmetiska operationer, så de går att skicka som inparametrar

class LA: # short for linear algebra

    def __init__(self, values):
        self.values = values
    
    # En statisk metod som en funktion du kan applicera
    # utan att behöva en instans av klassen.
    # därför behövs inte self som parameter, då det inte är
    # en instans av klassen, utan endast en funktion/metod till
    # klassen
    @staticmethod
    def toArray(_list):
        if type(_list) == list: return LA(_list)

    @staticmethod
    def sum(_list):
        if isinstance(_list, LA):
            return sum(_list.values)

    def __mul__(self, other):
        return self.__arithmetic(other, operator.mul)

    
    def __add__(self, other):
        return self.__arithmetic(other, operator.add)

    
    def __sub__(self, other):
        return self.__arithmetic(other, operator.sub)

    # Privat metod som endast kan användas inom klassen
    def __arithmetic(self, other, operator):
        if len(self) == len(other):
            values = zip(self.values, other.values)
            newValues = [operator(num[0],num[1]) for num in values]
            return LA(newValues)

    def size(self):
        return len(self)

    def __len__(self):
        return len(self.values)

    def __repr__(self):
        return f'<LA-array {self.values}>'

vectorOne = LA.toArray([1,2,0,4])
print(vectorOne)

vectorThree = vectorOne * vectorOne
print(vectorThree)
print(LA.sum(vectorThree))



In [ ]:
# Klasser exempel #5 - Laboration 5

class Country:

    def __init__(self, name, area, population):
        self.name = name
        self.area = float(area)
        self.population = int(population)
        self.density = self.getDensity()

    def getDensity(self):
        return round(self.population / self.area, 1)

    def __str__(self):
        return f'{self.name:<20} {self.area:<12} {self.population:<15} {self.density}'
    
    def __repr__(self):
        return f'<Country: {self.name}>'

# Detta kallas för en wrapper. Går att tänka som att du lägger
# till extra funktionalitet till något redan existerande
# som i detta fall är det en wrapper till open-funktionaliteten
class Europe:
    def __init__(self, fileName):
        try:
            self.f = open(fileName, mode='r', encoding='utf-8')
            print(f'Country{"":>13} Area{"":>8} Population {"":>4} Density\n')
            self.countries = []
        except FileNotFoundError:
            return

    # magic method som används utav "with"
    def __enter__(self):
        for c in self.f.readlines():
            name, area, pop = c.split(',')
            name, area, pop = name.strip(), area.strip(), pop.strip()
            self.countries.append(Country(name, area, pop))
        return self.countries

    # magic method som används utav "with"
    def __exit__(self, exc_type, exc_val, traceback):
        try:
            self.f.close()
        except AttributeError:
            return True

with Europe('./static/europa.txt') as eu:

    # sorterar eu-listan med hjälp av lambda
    eu = sorted(eu, key=lambda c: c.density)

    for country in eu: print(country)

## lambda
### Vad är lambda?
Lambda är ett sätt att skriva en funktion, oftast på en rad.

Skillnaden mellan en vanlig funktion och `lambda` är att `lambda` endast har ett `expression` (vad som returnas)

Du skriver en lambda funktion på följande vis:

`lambda` *arguments* : *expression*

In [ ]:
# lambda exempel #1

# det funktions-ekvivalenta
def f_add(a, b): return a + b

l_add = lambda a, b : a + b

if f_add(5,5) == l_add(5,5): print('They work the same')



### Vad används lambda för?

`Lambda` funktioner skapar `hemliga` funktioner. Det går att tänka sig att dessa hemliga funktioner är till för att hjälpa huvudfunktionen, men inte ska användas ensamt.

In [ ]:
# lambda exempel #2

addOne = lambda x : x + 1

numbers = [10, 4, 5, 8]

# map är huvudfunktionen, lambda är hjälpfunktionen
newNumbers = list(map(addOne, numbers))


In [ ]:
# lambda exempel #3

with Europe('./static/europa.txt') as eu:
    areaFilter = lambda c : c.area < 5000
    smallCountries = list(filter(areaFilter, eu))
    for country in smallCountries:
        print(country)

### Lambda är inte magiskt!

Du kan i många av fallen använda vanliga funktioner istället för lambda, men det är bra att förstå poängen med lambda, vilket är att det endast ska användas inuti specifika funktioner.

In [ ]:
# lambda exempel #3 - utan lambda

# problemet med denna approach är att du kan återanvända funktionen utanför användningsområdet den är tänkt för
def areaFilterFunc(c): return c.area < 5000

with Europe('./static/europa.txt') as eu:
    smallCountries = list(filter(areaFilterFunc, eu))
    for country in smallCountries:
        print(country)

In [ ]:
# lambda exempel #4
from helpers.coordinates import Stockholm # egen gjord modul som liknar Europe-klassen

lat, lon = 59.3466893, 18.0738567

with Stockholm('./static/map.json') as sthlm:
    streets = sthlm.getStreets()
    distance = lambda s : s.computeDistance(lat, lon)

    streetsNear = list(map(distance, streets))
    streetsNear.sort(key = lambda s : s.distance)
    for street in streetsNear[:10]:
        print(street)



## Felhantering

### Vad är det nödvändigt för?

Utan felhantering kan ditt första python-program se ut som en riktig __spagetti-kod__ om du ska försöka parera alla olika fel genom if-satser.

Ett mer *korrekt* och programatiskt sätt är att göra det med *`try`*/*`except`* satser. Det magiska med detta är att alla "fel" fångas upp av except som försöker exekveras i try-satsen

In [ ]:
# Felhantering exempel #1

try:
    print(42 / 0)
except Exception as exc:
    print(exc)

Du skriver den kod du vill `försöka` exekvera i try-satsen,
sedan fångar du upp alla möjliga fel med except-satsen.

I exempel #1 görs detta genom det generella fallet `exception` som är __basfallet__ för alla fel.

När man skriver `except X as Y` så får man felmeddelandet i Y-variabeln.

In [ ]:
# Felhantering exempel #2 - med klasser
from enum import Enum

# Enum är ett sätt att programatiskt ange statiska key-value-pairs 
class grade(Enum):
    A = 5
    B = 4.5
    C = 4
    D = 3.5
    E = 3
    F = 0

# Egengjord exception som byggs på från basfallet exception
class NotTeacher(Exception):
    pass

# En simpel User class, där man kan vara lärare eller inte
class User:
    def __init__(self, name, teacher = False):
        self.name = name
        self.teacher = teacher
        self.grade = None

    def isTeacher(self):
        return self.teacher
    
    def setGrade(self, g):
        self.grade = g
    
    def getGrade(self):
        return self.grade
    
    def __repr__(self):
        return f'<User name: {self.name}, grade: {self.grade}>'



users = [User('John', True), User('Alice')]

def gradeUser(teacher, student):
    try:
        if not teacher.isTeacher(): raise NotTeacher(f'{teacher.name} is not a Teacher')
        student.setGrade(grade.A)
    except Exception as exc:
        print(exc)

gradeUser(users[0], users[1])
print(users)
gradeUser(users[1], users[0])
print(users)

